# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Path to the csv file
path = "../output_data/cities.csv"
# Read the CSV file and create a dataframe
data_df = pd.read_csv(path)
data_df

,City ID,City,Country,Latitude,Longitude,Max. Temp (F),Humidity,Cloudiness,Wind Speed,Date
0,1,Pandan,PH,14.0453,124.1694,84.88,68,55,3.33,1643512262
1,2,Tuktoyaktuk,CA,69.4541,-133.0374,-6.07,96,100,16.24,1643512262
2,3,Vila Franca do Campo,PT,37.7167,-25.4333,56.19,69,78,10.47,1643512238
3,4,Remontnoye,RU,46.5614,43.6501,32.49,93,94,15.41,1643512263
4,5,Bubaque,GW,11.2833,-15.8333,78.21,59,40,12.26,1643512263
...,...,...,...,...,...,...,...,...,...,...
573,574,Roald,NO,62.5833,6.1333,41.05,81,75,21.85,1643512543
574,575,Wana,PK,32.2989,69.5725,44.71,49,17,3.91,1643512544
575,576,St. George's,GD,12.0564,-61.7485,76.62,75,37,10.94,1643512544
576,577,Sept-Îles,CA,50.2001,-66.3821,9.05,58,100,8.01,1643512545


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)
# Store latitude and longitude in locations
locations = data_df[["Latitude", "Longitude"]].astype(float)
# Store humidity values for use in heatmap
humidity = data_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Remove any rows with null values.
locations_data_df = data_df.dropna()
# Display locations dataframe
locations_data_df

,City ID,City,Country,Latitude,Longitude,Max. Temp (F),Humidity,Cloudiness,Wind Speed,Date
0,1,Pandan,PH,14.0453,124.1694,84.88,68,55,3.33,1643512262
1,2,Tuktoyaktuk,CA,69.4541,-133.0374,-6.07,96,100,16.24,1643512262
2,3,Vila Franca do Campo,PT,37.7167,-25.4333,56.19,69,78,10.47,1643512238
3,4,Remontnoye,RU,46.5614,43.6501,32.49,93,94,15.41,1643512263
4,5,Bubaque,GW,11.2833,-15.8333,78.21,59,40,12.26,1643512263
...,...,...,...,...,...,...,...,...,...,...
573,574,Roald,NO,62.5833,6.1333,41.05,81,75,21.85,1643512543
574,575,Wana,PK,32.2989,69.5725,44.71,49,17,3.91,1643512544
575,576,St. George's,GD,12.0564,-61.7485,76.62,75,37,10.94,1643512544
576,577,Sept-Îles,CA,50.2001,-66.3821,9.05,58,100,8.01,1643512545


In [6]:
# Find the ideal vacation locations

# Filter for locations with max temperature lower than 80 degrees but higher than 70, and save to a new dataframe
good_temp_df = locations_data_df.loc[(locations_data_df["Max. Temp (F)"] > 70) & 
                                     (locations_data_df["Max. Temp (F)"] < 80), :]
# Filter for locations with wind speed less than 10 mph, and save to a new dataframe
slight_breeze_df = good_temp_df.loc[(good_temp_df["Wind Speed"] < 10), :]
# Filter for locations with zero chance of cloudiness, and save to a new dataframe
clear_skies_df = slight_breeze_df.loc[(slight_breeze_df["Cloudiness"] == 0), :]
# Filter for locations with humidity lower than or equal to 70, and save to a new dataframe
ideal_spots_df = clear_skies_df.loc[(clear_skies_df["Humidity"] <= 70), :]
# Display ideal vacation locations
ideal_spots_df.head() 

,City ID,City,Country,Latitude,Longitude,Max. Temp (F),Humidity,Cloudiness,Wind Speed,Date
35,36,Kapaa,US,22.0752,-159.3190,79.12,69,0,1.01,1643512275
182,183,Road Town,VG,18.4167,-64.6167,73.99,66,0,7.29,1643512393
447,448,Cabo San Lucas,MX,22.8909,-109.9124,72.52,66,0,5.99,1643512339


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Saving a copy of the dataframe to hotel_df
hotel_df = ideal_spots_df.copy()
# Add a "Hotel Name" column to the DataFrame
hotel_df[["Hotel Name", "Address", "Lat", "Lng"]] = ""

In [8]:
# Use city latitude, longitude to use places API to find nearest hotel
row_counter = len(hotel_df)

for x in range(0, row_counter):
    latitude = hotel_df.iloc[x, 3]
    longitude = hotel_df.iloc[x, 4]
    target_search = "hotel"
    target_radius = 5000
    # set up a parameters dictionary
    params = {
        "location": f"{latitude},{longitude}",
        "keyword": target_search,
        "radius": target_radius,
        "key": g_key
    }
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    # convert response to json
    hotels_json = response.json()
    # Print the name and address of the first restaurant that appears
    hotel_df.iloc[x, 10] = hotels_json["results"][0]["name"]
    hotel_df.iloc[x, 11] = hotels_json["results"][0]["vicinity"]
    hotel_df.iloc[x, 12] = hotels_json["results"][0]["geometry"]["location"]["lat"]
    hotel_df.iloc[x, 13] = hotels_json["results"][0]["geometry"]["location"]["lng"]

In [9]:
# Preview dataframe with hotel information
hotel_df

,City ID,City,Country,Latitude,Longitude,Max. Temp (F),Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Address,Lat,Lng
35,36,Kapaa,US,22.0752,-159.3190,79.12,69,0,1.01,1643512275,Hotel Coral Reef,"4-1516 Kuhio Hwy, Kapaʻa",22.078352,-159.314819
182,183,Road Town,VG,18.4167,-64.6167,73.99,66,0,7.29,1643512393,Maria's By The Sea Hotel,"C9CM+983, Road Town",18.420881,-64.616707
447,448,Cabo San Lucas,MX,22.8909,-109.9124,72.52,66,0,5.99,1643512339,Solmar Resort,"Av Solmar #1, Centro, Cabo San Lucas",22.876556,-109.903533


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
                             info_box_content = hotel_info)

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))